# Evolution of Einstein's equations

# Initial data , setting length of the domain and discretization


In [256]:
# Parameters
m = 3
A = 0.05#0.1124921875#0.125
run = 1

1

In [257]:
res=m;
N=2.0^m*1000.0#2.0^m*1000.0;#2.0^m*500.0;#N=2.0^m*500.0#2.0^m*100.0;
Xf=1.0;

dx=Xf/N;
dt=0.8*round(dx,digits=10);
Nt=2.0^m*1000.0#100.0*2^m*10
Tf=Nt*dt; #final time
print(Tf)

0.8

In [258]:
print(N)

8000.0

In [259]:
println("dx =", dx)
println("dt =", dt)
println("dx/dt =", dx/dt)

dx =0.000125
dt =0.0001
dx/dt =1.25


In [260]:
# Setting RESOLUTION

global dir = "/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/res$res"
#global dir = "/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/res"

"/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/res3"

# Grid

In [261]:
ori=0.0;
initX1 = nothing
initX1=range(ori, stop=Xf, step=dx);

In [262]:
initX = range(round(ori-3.0*dx,digits=10), stop=Xf+3.0*dx, step=dx)

L=length(initX);
println("step size is  ", dx)

#T=range(dt,stop=Tf,step=dt)
#T=range(0,stop=Tf,step=dt)
println("the time step is ", dt)

step size is  0.000125
the time step is 0.0001


In [263]:
include("./ScalarField.jl");

# Initial data of m β Ψ and Ψ,x

In [264]:
using PyCall   
using Interpolations
using TickTock
using Dierckx

tick()
scipy = pyimport("scipy")
scipyinterpolate = pyimport("scipy.interpolate")


####

initm=zeros(L);
initbeta=zeros(L);
initpsi=zeros(L);
initderpsi=zeros(L);

state_array=[initm initbeta initpsi initderpsi initX];

#PSI
r0=0.7#0.3
sigma=0.3
initpsi[4:L-3] = init_gaussian(initX1,r0,sigma,A)

state_array[:,3] = initpsi
state_array = ghost(state_array)

#PSI,X FROM PSI
"""for i in 4:L-3
    initderpsi[i]=Der(state_array[:,:],i,3,initX)
end

state_array[:,4] = initderpsi
state_array = ghost(state_array)"""

#new
initderpsi[4:L-3] = init_gaussian_der(initX1,r0,sigma,A)
state_array[:,4] = initderpsi
state_array=ghost(state_array)

####


psi_func = Spline1D(initX[4:L-3], state_array[4:L-3,3],  k=4)
derpsi_func = Spline1D(initX[4:L-3], initderpsi[4:L-3],  k=4)#new

funcs = [psi_func, derpsi_func];

tock()

[ Info:  started timer at: 2023-04-05T15:35:40.567
[ Info:          0.440305026s: 440 milliseconds


In [265]:
#BETA
beta0=0
initbeta[4:L-3]=rk4wrapper(SFconstraint_beta,beta0,initX1,0,funcs)
state_array[:,2]=initbeta;
state_array=ghost(state_array);


#M
m0=0
initm[4:L-3]=rk4wrapper(SFconstraint_m,m0,initX1,0,funcs)
state_array[:,1]=initm;
state_array = ghost(state_array);

In [266]:
"""using Plots #showing initial data and the initial status of the horizon finder
plot(initX, state_array[:,1:4], xaxis="x",title="Initial Data", labels=["m" "beta" "psi" "psi,x"])"""

"using Plots #showing initial data and the initial status of the horizon finder\nplot(initX, state_array[:,1:4], xaxis=\"x\",title=\"Initial Data\", labels=[\"m\" \"beta\" \"psi\" \"psi,x\"])"

# Defining the initial state array for the evolution m β Ψ and Ψ,x

In [267]:
using CSV, Tables
#CSV.write(dir*"/time_step0.csv", Tables.table(transpose(Matrix(state_array))), writeheader=false)
CSV.write(dir*"/time_step0.csv", Tables.table(state_array), writeheader=false)

"/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/res3/time_step0.csv"

In [268]:
timestep=0
criticality=0.0
explode=0.0
evol_stats = [run criticality A sigma r0 timestep explode]
monitor_ratio = zeros(L)
CSV.write("/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/parameters.csv", Tables.table(evol_stats), writeheader=true, header=["run", "criticality", "A", "sigma", "r0", "timestep", "explode"])

"/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/parameters.csv"

In [269]:
ginit=dt_scale(initX,state_array[:,1],state_array[:,2],dx)
println(update_dt(initX,state_array[:,1],state_array[:,2],dx,ginit)/dt)


1.25


 ## Time evolution

fixed grid

In [270]:
finaltime=0.1#length(T)-1
T = timeevolution(state_array,finaltime,dir,dt,run);

iteration 1 dt is 0.0001, time of iteration is 0.0001
iteration 2 dt is 0.0001, time of iteration is 0.0002
iteration 3 dt is 0.0001, time of iteration is 0.00030000000000000003
iteration 4 dt is 0.0001, time of iteration is 0.0004
iteration 5 dt is 0.0001, time of iteration is 0.0005
iteration 6 dt is 0.0001, time of iteration is 0.0006000000000000001
iteration 7 dt is 0.0001, time of iteration is 0.0007000000000000001
iteration 8 dt is 0.0001, time of iteration is 0.0008000000000000001
iteration 9 dt is 0.0001, time of iteration is 0.0009000000000000002
iteration 10 dt is 0.0001, time of iteration is 0.0010000000000000002
iteration 11 dt is 0.0001, time of iteration is 0.0011000000000000003
iteration 12 dt is 0.0001, time of iteration is 0.0012000000000000003
iteration 13 dt is 0.0001, time of iteration is 0.0013000000000000004
iteration 14 dt is 0.0001, time of iteration is 0.0014000000000000004
iteration 15 dt is 0.0001, time of iteration is 0.0015000000000000005
iteration 16 dt is

iteration 122 dt is 0.0001, time of iteration is 0.012199999999999982
iteration 123 dt is 0.0001, time of iteration is 0.012299999999999981
iteration 124 dt is 0.0001, time of iteration is 0.01239999999999998
iteration 125 dt is 0.0001, time of iteration is 0.01249999999999998
iteration 126 dt is 0.0001, time of iteration is 0.01259999999999998
iteration 127 dt is 0.0001, time of iteration is 0.012699999999999979
iteration 128 dt is 0.0001, time of iteration is 0.012799999999999978
iteration 129 dt is 0.0001, time of iteration is 0.012899999999999977
iteration 130 dt is 0.0001, time of iteration is 0.012999999999999977
iteration 131 dt is 0.0001, time of iteration is 0.013099999999999976
iteration 132 dt is 0.0001, time of iteration is 0.013199999999999976
iteration 133 dt is 0.0001, time of iteration is 0.013299999999999975
iteration 134 dt is 0.0001, time of iteration is 0.013399999999999974
iteration 135 dt is 0.0001, time of iteration is 0.013499999999999974
iteration 136 dt is 0.0

iteration 240 dt is 0.0001, time of iteration is 0.02399999999999991
iteration 241 dt is 0.0001, time of iteration is 0.02409999999999991
iteration 242 dt is 0.0001, time of iteration is 0.02419999999999991
iteration 243 dt is 0.0001, time of iteration is 0.02429999999999991
iteration 244 dt is 0.0001, time of iteration is 0.024399999999999908
iteration 245 dt is 0.0001, time of iteration is 0.024499999999999907
iteration 246 dt is 0.0001, time of iteration is 0.024599999999999907
iteration 247 dt is 0.0001, time of iteration is 0.024699999999999906
iteration 248 dt is 0.0001, time of iteration is 0.024799999999999905
iteration 249 dt is 0.0001, time of iteration is 0.024899999999999905
iteration 250 dt is 0.0001, time of iteration is 0.024999999999999904
iteration 251 dt is 0.0001, time of iteration is 0.025099999999999904
iteration 252 dt is 0.0001, time of iteration is 0.025199999999999903
iteration 253 dt is 0.0001, time of iteration is 0.025299999999999902
iteration 254 dt is 0.00

iteration 358 dt is 0.0001, time of iteration is 0.0358
iteration 359 dt is 0.0001, time of iteration is 0.0359
iteration 360 dt is 0.0001, time of iteration is 0.036000000000000004
iteration 361 dt is 0.0001, time of iteration is 0.03610000000000001
iteration 362 dt is 0.0001, time of iteration is 0.03620000000000001
iteration 363 dt is 0.0001, time of iteration is 0.03630000000000001
iteration 364 dt is 0.0001, time of iteration is 0.036400000000000016
iteration 365 dt is 0.0001, time of iteration is 0.03650000000000002
iteration 366 dt is 0.0001, time of iteration is 0.03660000000000002
iteration 367 dt is 0.0001, time of iteration is 0.036700000000000024
iteration 368 dt is 0.0001, time of iteration is 0.03680000000000003
iteration 369 dt is 0.0001, time of iteration is 0.03690000000000003
iteration 370 dt is 0.0001, time of iteration is 0.03700000000000003
iteration 371 dt is 0.0001, time of iteration is 0.037100000000000036
iteration 372 dt is 0.0001, time of iteration is 0.03720

iteration 477 dt is 0.0001, time of iteration is 0.04770000000000034
iteration 478 dt is 0.0001, time of iteration is 0.04780000000000034
iteration 479 dt is 0.0001, time of iteration is 0.047900000000000345
iteration 480 dt is 0.0001, time of iteration is 0.04800000000000035
iteration 481 dt is 0.0001, time of iteration is 0.04810000000000035
iteration 482 dt is 0.0001, time of iteration is 0.048200000000000354
iteration 483 dt is 0.0001, time of iteration is 0.04830000000000036
iteration 484 dt is 0.0001, time of iteration is 0.04840000000000036
iteration 485 dt is 0.0001, time of iteration is 0.04850000000000036
iteration 486 dt is 0.0001, time of iteration is 0.048600000000000365
iteration 487 dt is 0.0001, time of iteration is 0.04870000000000037
iteration 488 dt is 0.0001, time of iteration is 0.04880000000000037
iteration 489 dt is 0.0001, time of iteration is 0.048900000000000374
iteration 490 dt is 0.0001, time of iteration is 0.04900000000000038
iteration 491 dt is 0.0001, ti

iteration 596 dt is 0.0001, time of iteration is 0.05960000000000068
iteration 597 dt is 0.0001, time of iteration is 0.05970000000000068
iteration 598 dt is 0.0001, time of iteration is 0.059800000000000686
iteration 599 dt is 0.0001, time of iteration is 0.05990000000000069
iteration 600 dt is 0.0001, time of iteration is 0.06000000000000069
iteration 601 dt is 0.0001, time of iteration is 0.060100000000000695
iteration 602 dt is 0.0001, time of iteration is 0.0602000000000007
iteration 603 dt is 0.0001, time of iteration is 0.0603000000000007
iteration 604 dt is 0.0001, time of iteration is 0.0604000000000007
iteration 605 dt is 0.0001, time of iteration is 0.060500000000000706
iteration 606 dt is 0.0001, time of iteration is 0.06060000000000071
iteration 607 dt is 0.0001, time of iteration is 0.06070000000000071
iteration 608 dt is 0.0001, time of iteration is 0.060800000000000715
iteration 609 dt is 0.0001, time of iteration is 0.06090000000000072
iteration 610 dt is 0.0001, time 

iteration 715 dt is 0.0001, time of iteration is 0.07150000000000102
iteration 716 dt is 0.0001, time of iteration is 0.07160000000000102
iteration 717 dt is 0.0001, time of iteration is 0.07170000000000103
iteration 718 dt is 0.0001, time of iteration is 0.07180000000000103
iteration 719 dt is 0.0001, time of iteration is 0.07190000000000103
iteration 720 dt is 0.0001, time of iteration is 0.07200000000000104
iteration 721 dt is 0.0001, time of iteration is 0.07210000000000104
iteration 722 dt is 0.0001, time of iteration is 0.07220000000000104
iteration 723 dt is 0.0001, time of iteration is 0.07230000000000104
iteration 724 dt is 0.0001, time of iteration is 0.07240000000000105
iteration 725 dt is 0.0001, time of iteration is 0.07250000000000105
iteration 726 dt is 0.0001, time of iteration is 0.07260000000000105
iteration 727 dt is 0.0001, time of iteration is 0.07270000000000106
iteration 728 dt is 0.0001, time of iteration is 0.07280000000000106
iteration 729 dt is 0.0001, time o

iteration 834 dt is 0.0001, time of iteration is 0.08340000000000136
iteration 835 dt is 0.0001, time of iteration is 0.08350000000000136
iteration 836 dt is 0.0001, time of iteration is 0.08360000000000137
iteration 837 dt is 0.0001, time of iteration is 0.08370000000000137
iteration 838 dt is 0.0001, time of iteration is 0.08380000000000137
iteration 839 dt is 0.0001, time of iteration is 0.08390000000000138
iteration 840 dt is 0.0001, time of iteration is 0.08400000000000138
iteration 841 dt is 0.0001, time of iteration is 0.08410000000000138
iteration 842 dt is 0.0001, time of iteration is 0.08420000000000138
iteration 843 dt is 0.0001, time of iteration is 0.08430000000000139
iteration 844 dt is 0.0001, time of iteration is 0.08440000000000139
iteration 845 dt is 0.0001, time of iteration is 0.0845000000000014
iteration 846 dt is 0.0001, time of iteration is 0.0846000000000014
iteration 847 dt is 0.0001, time of iteration is 0.0847000000000014
iteration 848 dt is 0.0001, time of i

iteration 953 dt is 0.0001, time of iteration is 0.0953000000000017
iteration 954 dt is 0.0001, time of iteration is 0.0954000000000017
iteration 955 dt is 0.0001, time of iteration is 0.09550000000000171
iteration 956 dt is 0.0001, time of iteration is 0.09560000000000171
iteration 957 dt is 0.0001, time of iteration is 0.09570000000000171
iteration 958 dt is 0.0001, time of iteration is 0.09580000000000172
iteration 959 dt is 0.0001, time of iteration is 0.09590000000000172
iteration 960 dt is 0.0001, time of iteration is 0.09600000000000172
iteration 961 dt is 0.0001, time of iteration is 0.09610000000000173
iteration 962 dt is 0.0001, time of iteration is 0.09620000000000173
iteration 963 dt is 0.0001, time of iteration is 0.09630000000000173
iteration 964 dt is 0.0001, time of iteration is 0.09640000000000173
iteration 965 dt is 0.0001, time of iteration is 0.09650000000000174
iteration 966 dt is 0.0001, time of iteration is 0.09660000000000174
iteration 967 dt is 0.0001, time of 

## Movie of the evolution

In [218]:
using Plots, CSV, DataFrames
import GR
anim = @animate for n in 1:10:740#length(T)-1
    
    time_step=T[n+1]-T[n];
    IJulia.clear_output(true)
    
    functions3=DataFrame(CSV.File(dir*"/time_step$n.csv",header=false, delim=","))
    plot()
        plot1=functions3.Column1[4:L-3];
        plot2=functions3.Column2[4:L-3];
        plot3=functions3.Column3[4:L-3];
        plot4=functions3.Column4[4:L-3];
        plot5=functions3.Column5[4:L-3];
    d=round(T[n],digits=1)
    #plot!(X1, [plot1 plot2 plot3 plot4], labels=["m" "beta" "psi" "psi,x"],ylim=[-0.1,0.1], xlim=[0, 0.1]) |> display 
    plot!(initX[4:L-3], [plot1 plot2 plot3 plot4], labels=["m" "beta" "psi" "psi,x"],ylim=[-1,1])#, xlim=[0, 0.1]) |> display 
    
    end

gif(anim, "anim_res$res.gif", fps = 1)

LoadError: ArgumentError: "/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/res3/time_step1.csv" is not a valid file or doesn't exist

In [219]:
dir

"/home/rita13santos/Desktop/MSc Thesis/Git/ScalarField/DATA/res3"